In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
submission = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/sample_submission.csv')
train = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/train.csv',
                    parse_dates=['date_time'], infer_datetime_format=True)
test = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/test.csv',
                    parse_dates=['date_time'], infer_datetime_format=True)
train.info()

In [ ]:
train['hour'] = train['date_time'].dt.hour
test['hour'] = test['date_time'].dt.hour
train['dayofweek'] = train['date_time'].dt.dayofweek
test['dayofweek'] = test['date_time'].dt.dayofweek

In [ ]:
y1 = train.target_carbon_monoxide
y2 = train.target_benzene
y3 = train.target_nitrogen_oxides

# scale the targets
import numpy as np
y1_train = np.log1p(y1)
y2_train=np.log1p(y2)
y3_train=np.log1p(y3)

In [ ]:
n=6
target='target_benzene'

train['lag_%s'%n]=train[target].shift(n)
test['lag_%s'%n]=pd.Series(list())
col4train=['deg_C', 'relative_humidity', 'absolute_humidity',
       'sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5',
       
       'hour', 'dayofweek', 'lag_%s'%n]
X=train[col4train]
x=test[col4train]
#x=x[:20]
import xgboost as xgb
model = xgb.XGBRegressor(objective='reg:squaredlogerror', n_estimators=200, learning_rate=0.2)
model.fit(X,np.log1p(train[target]))
          
x['lag_%s'%n]=pd.Series(list(X.iloc[-n:,10]))

preds=[]
    
for i in range(0,len(x),n):
    
    #print(i)
    pred=model.predict(x.iloc[i:i+n,:])
    #print(pred)
    preds=preds+list(pred) 
    if i+2*n<=len(x):
        x.iloc[i+n:i+2*n,10]=pred
        #print(x)
    else:
        x.iloc[len(x)-len(x)%n:,10]=pred[:len(x)%n]
        pred=model.predict(x.iloc[len(x)-len(x)%n:,:])
        


In [ ]:
test['lag_%s'%n]=pd.Series(preds)


In [ ]:
test[['date_time','lag_%s'%n]].to_csv('lag_%s_y2.csv'%n)